# gpx parser

In [2]:
import gpxpy
import gpxpy.gpx

In [30]:
# Parsing an existing file:
# -------------------------

gpx_file = open('test.gpx', 'r')

gpx = gpxpy.parse(gpx_file, version="1.0")


In [61]:
for track in gpx.tracks:
    for segment in track.segments:
        for idx,point in enumerate(segment.points):
            #print(point.time)
            #print(idx)
            if idx < 10:
                print('Point at ({0},{1}) -> {2}'.format(point.latitude, point.longitude, point.elevation));


Point at (47.4504,12.3352) -> 811.0999755859375
Point at (47.4501,12.3354) -> 808.5999755859375
Point at (47.4499,12.3355) -> 809.2000122070312
Point at (47.4497,12.3356) -> 812.5999755859375
Point at (47.4495,12.3358) -> 816.4000244140625
Point at (47.4493,12.3359) -> 819.4000244140625
Point at (47.4492,12.336) -> 820.7000122070312
Point at (47.449,12.3362) -> 820.9000244140625
Point at (47.4489,12.3363) -> 822.0
Point at (47.4487,12.3364) -> 824.7999877929688


In [31]:
p0 = gpx.tracks[0].segments[0].points[0]
p1 = gpx.tracks[0].segments[0].points[1]
print(p0)
print(p1)

[trkpt:47.4504,12.3352@811.0999755859375@2023-01-01 07:34:19+00:00]
[trkpt:47.4501,12.3354@808.5999755859375@2023-01-01 07:34:57+00:00]


In [35]:
moving_data = gpx.get_moving_data(raw=True)
print(moving_data)

MovingData(moving_time=13491.0, stopped_time=5545.0, moving_distance=69257.70288075981, stopped_distance=237.0913580280757, max_speed=22.706002111137018)


In [75]:
from gpxplotter import read_gpx_file, create_folium_map, add_segment_to_map

# Define some properties for drawing the line:
line_options_up = {'color': 'red', 'weight': 3, 'opacity': 0.4}
line_options_down = {'color': 'green', 'weight': 6, 'opacity': 0.7}

#the_map = create_folium_map(tiles='kartverket_topo4')
the_map = create_folium_map(tiles='stamenterrain')

for track in read_gpx_file('test.gpx'):
    for i, segment in enumerate(track['segments']):
        print(i)
        if elev_diff[i] < 0:
            add_segment_to_map(the_map, segment, line_options=line_options_up)
        else:
            add_segment_to_map(the_map, segment, line_options=line_options_down)

# To store the map as a HTML page:
# the_map.save('map_000.html')

# To display the map in a Jupyter notebook:
the_map
the_map.save('map_000.html')

0


In [72]:

def calc_elevation_difference():
    #up or down
    points = gpx.tracks[0].segments[0].points
    print(len(points))
    total = len(points)
    elev_diff.append(0)
    for idp,p in enumerate(points):
        diff = p.elevation - points[idp-1].elevation
        if idp < 10:
            print(p)
            print(diff)
        if idp > 0 and idp < total:
            elev_diff.append(diff)
            #elev_diff 
         

In [73]:
#https://stackoverflow.com/questions/45840118/how-do-i-calculate-speed-from-a-gpx-file-if-the-speed-tag-itself-is-not-given
elev_diff = []
calc_elevation_difference()
#print(elev_diff)
print(len(elev_diff))

2699
[trkpt:47.4504,12.3352@811.0999755859375@2023-01-01 07:34:19+00:00]
14.0
[trkpt:47.4501,12.3354@808.5999755859375@2023-01-01 07:34:57+00:00]
-2.5
[trkpt:47.4499,12.3355@809.2000122070312@2023-01-01 07:35:01+00:00]
0.60003662109375
[trkpt:47.4497,12.3356@812.5999755859375@2023-01-01 07:35:06+00:00]
3.39996337890625
[trkpt:47.4495,12.3358@816.4000244140625@2023-01-01 07:35:10+00:00]
3.800048828125
[trkpt:47.4493,12.3359@819.4000244140625@2023-01-01 07:35:14+00:00]
3.0
[trkpt:47.4492,12.336@820.7000122070312@2023-01-01 07:35:18+00:00]
1.29998779296875
[trkpt:47.449,12.3362@820.9000244140625@2023-01-01 07:35:22+00:00]
0.20001220703125
[trkpt:47.4489,12.3363@822.0@2023-01-01 07:35:26+00:00]
1.0999755859375
[trkpt:47.4487,12.3364@824.7999877929688@2023-01-01 07:35:30+00:00]
2.79998779296875
2699
